This workflow was generated to build a combined political boundary dataset for North America in order to find coverage of EPA ecoregion units with those areas of interest. That follows a pattern that User:TimK set in WikiData of including US State and County links. This is interesting in the WikiData sense as it will further build out linked data in a potentially useful way, allowing simple graph tracing through ecoregion/state links. Linkages to US States could be derived from the EPA data that includes state boundaries. However, I thought it would be interesting to use the geometry from the ecoregions themselves against reasonable current data sources to generate these links and include Canadian and Mexican states/provinces as well for more thorough coverage of North America. In future, it would be interesting to explore linking the ecoregion concepts to lower level political boundaries in the other two countriles and other relevant linkable items in WikiData such as Native American/First Nations lands and territories. 

I tried to find linkable and reasonably authoritative sources for state/province boundaries from each country but was only partially successful (mostly because my Spanish isn't as good as it should be). I did pull the combined set of boundaries used by the CEC that did the work on documenting these ecoregions, but the provenance and usability of that source is perhaps not the best and a brief comparison with more authoritative boundary options showed varying results. I had to rely on one manual download for Mexican states, but got the rest through a process that I can run from URL through to cleanup with a set of functions I built for the purpose.

The spatial process to find all the intersecting boundaries between the datasets is fairly intensive, and it seemed like there should be a way to load these data into one or another cloud resource to get that done. I spent some time experimenting with how I might run this with Google BigQuery as Google has a public data source from US Census for states and counties and should well have other political boundaries as well, though I couldn't turn them up. I got stymied on getting my ecoregions loaded in order to have all the data together, and I'll have to spend some more time on where those problems were coming up. There's kind of a dearth of information online on how to properly format and check more complex geometry in order for it to load. It should have worked to convert everything to WKT and then load via a file from GoogleCloudStorage, but I kept running into errors with not enough information for me to pin it down. So, I ended up falling back to a local instance of PostGIS after trying and failing with Geopandas directly in my code.

In [10]:
from functions import get_source_df, lookup_admin_ref
import os
import copy

import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement

Since I had to repeat the same basic extract and transform operations multiple times on source data for both boundaries and ecoregions, I wrote some helper functions for downloading, extracting, loading, and some basic transformations on shapefile source data from either HTTP or FTP URLs. Because, in this use case, I ultimately need to check all ecoregion boundaries against all political boundaries, I simplified boundary data to a common schema containing name, a useful form of identifier (e.g., FIPS codes), abbreviation where applicable, and the geometry. This gives me everything I really need in order to check spatial intersections and find corresponding WikiData identifiers. I added in a country property to help filter records.

In [2]:
%%time
us_states = get_source_df(
    source_url="https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip",
    filter_to_columns = ["NAME", "GEOID", "STUSPS", "geometry"],
    add_columns=[('country', 'US')],
    rename_columns={"NAME": "name", "GEOID": "identifier", "STUSPS": "abbreviation"}
)

us_counties = get_source_df(
    source_url="https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip",
    filter_to_columns = ["NAME", "GEOID", "geometry"],
    add_columns=[('country', 'US')],
    rename_columns={"NAME": "name", "GEOID": "identifier", "STUSPS": "abbreviation"}
)

ca_states = get_source_df(
    source_url="http://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lpr_000b16a_e.zip",
    filter_to_columns = ["PRENAME", "PREABBR", "geometry"],
    add_columns=[('country', 'CA')],
    rename_columns={"PRENAME": "name", "PREABBR": "abbreviation"}
)

mx_states = get_source_df(
    target_file_name="mexstates.shp",
    filter_to_columns = ["ADMIN_NAME", "FIPS_ADMIN", "GMI_ADMIN", "geometry"],
    add_columns=[('country', 'MX')],
    rename_columns={"ADMIN_NAME": "name", "FIPS_ADMIN": "identifier", "GMI_ADMIN": "abbreviation"}
)

CPU times: user 1.88 s, sys: 319 ms, total: 2.2 s
Wall time: 2.44 s


I reproject every dataset to a common CRS for processing.

In [3]:
%%time
ca_states = ca_states.to_crs({'init': 'epsg:4326'})
us_states = us_states.to_crs({'init': 'epsg:4326'})
us_counties = us_counties.to_crs({'init': 'epsg:4326'})
mx_states = mx_states.to_crs({'init': 'epsg:4326'})

CPU times: user 7.27 s, sys: 232 ms, total: 7.5 s
Wall time: 7.68 s


For my purposes, it made sense to pull every source into one common dataset since they all now have the same schema and I can run my spatial operations against combined data.

In [4]:
%%time
boundaries = gpd.GeoDataFrame(pd.concat([ca_states, us_states, mx_states, us_counties], ignore_index=True, sort=False))
boundaries.crs = us_states.crs
boundaries = boundaries.where(pd.notnull(boundaries), None)

CPU times: user 14.1 ms, sys: 2.48 ms, total: 16.6 ms
Wall time: 16.4 ms


The last major thing that I can do on this side of data processing is to lookup identifiers from WikiData. The point here is to establish the actual identities to which we will link to as intersecting with our ecoregions (or whatever other type of relationship to these political entities we might establish), and it's useful to run this process first for later use. This can be done in a number of different ways using SPARQL queries to constrain the results. Each case is a little bit different between states/provinces in all three countries and US Counties. For US States/Counties and for MX States, we have an identifier that is used regularly in WikiData and we can look for matches on that explicit property and identifier values from source data. In the case of Canada, I needed to use a constraint to a one of two properties the types of political units in Canada from our source data. This process would take a very long time to run if we were to run individual SPARQL queries for every name and identifier we have. So, I have a function that runs a query for every item with the particular properties of an instance of the two types of political units in Canada so that we can simply consult that list. I add this new property to the data model using a function on the dataframe.

In [6]:
%%time
boundaries["admin_id"] = boundaries.apply(lambda row: lookup_admin_ref(row["country"], row["name"], row["identifier"], return_var="wd_id"), axis=1)
boundaries["country_id"] = boundaries.apply(lambda row: lookup_admin_ref(row["country"], row["name"], row["identifier"], return_var="country"), axis=1)

CPU times: user 27.3 s, sys: 1.11 s, total: 28.5 s
Wall time: 29.1 s


In [8]:
boundaries.to_file("data_cache/boundaries.gpkg", driver="GPKG")
#boundaries = gpd.read_file("data_cache/boundaries.gpkg")


In order to load into PostGIS, I needed to separate polygon from multipolygon in terms of geometry and convert to WKT in a "geom" field for convenience in loading to PostgreSQL. The following code makes a complete copy of the boundaries geodataframe to deal with indexing issues and then splits out the two datasets of like geometry.

In [11]:
%%time
boundaries_multi = boundaries.apply(copy.deepcopy)
boundaries_multi.drop(boundaries_multi[boundaries_multi.geometry.geom_type == "Polygon"].index, inplace=True)
boundaries.drop(boundaries[boundaries.geometry.geom_type == "MultiPolygon"].index, inplace=True)

boundaries_multi['geom'] = boundaries_multi['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
boundaries['geom'] = boundaries['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))

boundaries_multi.drop('geometry', 1, inplace=True)
boundaries.drop('geometry', 1, inplace=True)

CPU times: user 6.82 s, sys: 191 ms, total: 7.01 s
Wall time: 7.34 s


I used a local Postgres instance in a Docker container, but this can be pointed anywhere. Presumably, I could have figured out how to do this in something like Amazon RDS as well with Amazon's version of public data containing some of the boundary datasets I'd want to use. Longer term, some option like that would be useful as a more thorough system to support many cases where finding simple intersects with political boundaries would be desired.

In [12]:
db_connection_url = f"postgres://{os.environ.get('PG_USER')}:{os.environ.get('PG_PASS')}@{os.environ.get('PG_HOST')}:{os.environ.get('PG_PORT')}/ecoregions";
engine = create_engine(db_connection_url)

The last step in this sequence is to load up my two datasets of like geometry to PostgreSQL for further processing.

In [13]:
%%time
boundaries.to_sql(
    "boundaries_p", 
    engine, 
    if_exists='replace', 
    index=False,
    dtype={'geom': Geometry('POLYGON', srid=4326)}
)

boundaries_multi.to_sql(
    "boundaries_m", 
    engine, 
    if_exists='replace', 
    index=False,
    dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)}
)

CPU times: user 789 ms, sys: 453 ms, total: 1.24 s
Wall time: 18.9 s
